In [1]:
# install deps -- % makes sure they install in the current virtual env
%pip install transformers
%pip install torch torchvision torchaudio
%pip install llama-index

You should consider upgrading via the '/Users/alex.holachek/Desktop/tutorial/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/alex.holachek/Desktop/tutorial/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/alex.holachek/Desktop/tutorial/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/alex.holachek/Desktop/tutorial/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Create index


In [3]:
# to save money we're using the local embed model instead of the OpenAI default text-embedding-ada-002
# this will use a HuggingFace embedding model instead

from llama_index import (
    ServiceContext,
    set_global_service_context,
)

service_context = ServiceContext.from_defaults(embed_model="local")
set_global_service_context(service_context)

In [4]:
import json
import os.path

from llama_index import (
    StorageContext,
    VectorStoreIndex,
    download_loader,
    load_index_from_storage,
)

PERSIST_DIR = "./storage"
FORCE_REINDEX = True


def create_and_store_embeddings():
    # load wikipedia urls
    f = open("season_one_episodes.json")
    urls = json.load(f)

    BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
    loader = BeautifulSoupWebReader()
    documents = loader.load_data(urls=urls)
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
    return index


def load_existing_index():
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    return load_index_from_storage(storage_context)


if FORCE_REINDEX or not os.path.exists(PERSIST_DIR):
    index = create_and_store_embeddings()
else:
    index = load_existing_index()

### Create query engine


In [5]:
from llama_index.prompts import PromptTemplate

# this is just our local shared prompt
from prompt import base_prompt

text_qa_template_str = """
  Context information is provided below:
  ________________________________________
  {context_str}
  ________________________________________
  Using only the context information, answer the question: {query_str}
  If the context isn't helpful, say that you don't know the answer.
  """

text_qa_template = PromptTemplate(text_qa_template_str)

query_engine = index.as_query_engine(text_qa_template=text_qa_template)

response = query_engine.query(base_prompt)
print(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


- The episode begins with a flashback to 1992, where Chuck McGill meets Jimmy McGill in jail and agrees to represent him if he stops running cons and finds legitimate employment.
- In the present, Jimmy is anxious about Nacho Varga's plot to steal from the Kettlemans and warns them anonymously.
- The next morning, Jimmy finds out that the Kettleman family is missing and believes Nacho has kidnapped them.
- Jimmy is picked up by the police and Nacho admits to surveilling the Kettlemans but denies kidnapping them.
- Jimmy tries to convince the police to release Nacho but convinces Kim to take him to the Kettleman house to investigate.
- Jimmy notices inconsistencies and theorizes that the Kettlemans staged their kidnapping.
- At the courthouse, Jimmy starts a fight with Mike Ehrmantraut and the police ask Mike to press assault charges.
- Mike changes his mind after hearing Jimmy's theory and suggests that the Kettlemans are hiding somewhere close to home.
- Jimmy explores the desert near